In [3]:
!pip install gensim==4.3.3
!pip install matplotlib
!pip install scikit-learn

In [8]:

!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz


--2024-09-24 13:38:12--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.154.144.87, 18.154.144.102, 18.154.144.74, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.154.144.87|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2024-09-24 13:38:12--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.154.144.87, 18.154.144.102, 18.154.144.74, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.154.144.87|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [9]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

# English embeddings
en_embeddings = KeyedVectors.load_word2vec_format('cc.en.300.vec.gz', binary=False)

# Hindi embeddings
hi_embeddings = KeyedVectors.load_word2vec_format('cc.hi.300.vec.gz', binary=False)

In [4]:

!wget -c https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.0-5000.txt

!wget -c https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.5000-6500.txt

--2024-09-24 13:32:20--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.0-5000.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.25, 13.226.210.78, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219327 (214K) [text/x-c++]
Saving to: ‘en-hi.0-5000.txt’

en-hi.0-5000.txt    100%[===================>] 214.19K  --.-KB/s    in 0.03s   

2024-09-24 13:32:20 (7.82 MB/s) - ‘en-hi.0-5000.txt’ saved [219327/219327]

--2024-09-24 13:32:20--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.5000-6500.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.25, 13.226.210.78, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52464 (51K) [text/plain]
Saving to: ‘en-hi.5000-6500.txt’

en-hi.5000-6500

In [10]:


# For English
en_embeddings.vectors = en_embeddings.vectors[:100000]
en_embeddings.index_to_key = en_embeddings.index_to_key[:100000]
en_embeddings.key_to_index = {word: idx for idx, word in enumerate(en_embeddings.index_to_key)}

# For Hindi
hi_embeddings.vectors = hi_embeddings.vectors[:100000]
hi_embeddings.index_to_key = hi_embeddings.index_to_key[:100000]
hi_embeddings.key_to_index = {word: idx for idx, word in enumerate(hi_embeddings.index_to_key)}

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import KeyedVectors
import gc

In [12]:
MAX_VOCAB_SIZE = 100000  # Adjust based on available memory
EMBEDDING_DIM = 300
BATCH_SIZE = 64  # Adjust based on memory constraints
N_EPOCHS = 10  # Number of training epochs
K_CSLS = 10  # Number of nearest neighbors for CSLS

In [13]:
!wget -c https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.0-5000.txt
!wget -c https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.5000-6500.txt


--2024-09-24 14:24:51--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.0-5000.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.78, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2024-09-24 14:24:51--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.5000-6500.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.78, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [14]:
# Loading the bilingual lexicon
def load_bilingual_lexicon(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        bilingual_dict = [line.strip().split() for line in f]
    return bilingual_dict

# Loading training dictionary
train_dict = load_bilingual_lexicon('en-hi.0-5000.txt')

In [15]:
train_dict = load_bilingual_lexicon('en-hi.0-5000.txt')
test_dict = load_bilingual_lexicon('en-hi.5000-6500.txt')

In [16]:
!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz

--2024-09-24 14:25:03--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.78, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2024-09-24 14:25:03--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.78, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [17]:
from gensim.models import KeyedVectors


In [18]:
en_embeddings = KeyedVectors.load_word2vec_format('cc.en.300.vec.gz', binary=False)
hi_embeddings = KeyedVectors.load_word2vec_format('cc.hi.300.vec.gz', binary=False)


In [21]:
en_embeddings.vectors = en_embeddings.vectors[:MAX_VOCAB_SIZE]
en_embeddings.index_to_key = en_embeddings.index_to_key[:MAX_VOCAB_SIZE]
en_embeddings.key_to_index = {word: idx for idx, word in enumerate(en_embeddings.index_to_key)}

hi_embeddings.vectors = hi_embeddings.vectors[:MAX_VOCAB_SIZE]
hi_embeddings.index_to_key = hi_embeddings.index_to_key[:MAX_VOCAB_SIZE]
hi_embeddings.key_to_index = {word: idx for idx, word in enumerate(hi_embeddings.index_to_key)}

# Step 5: Prepare Embeddings for Training
# Convert embeddings to torch tensors
src_embeddings = torch.from_numpy(en_embeddings.vectors).float()
tgt_embeddings = torch.from_numpy(hi_embeddings.vectors).float()

# Normalize embeddings
src_embeddings = src_embeddings / src_embeddings.norm(dim=1, keepdim=True)
tgt_embeddings = tgt_embeddings / tgt_embeddings.norm(dim=1, keepdim=True)

# Move embeddings to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
src_embeddings = src_embeddings.to(device)
tgt_embeddings = tgt_embeddings.to(device)


In [22]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim=2048):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.layer(x)

In [23]:


# Initialize mapping and discriminator
mapping = nn.Linear(EMBEDDING_DIM, EMBEDDING_DIM, bias=False)
nn.init.eye_(mapping.weight)  # Initialize as identity matrix
discriminator = Discriminator(EMBEDDING_DIM)

# Move models to device
mapping = mapping.to(device)
discriminator = discriminator.to(device)

# Step 7: Set Up Optimizers
mapping_optimizer = optim.SGD(mapping.parameters(), lr=0.1)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.001, betas=(0.5, 0.999))

# Step 8: Adversarial Training Loop
num_embeddings = src_embeddings.size(0)
for epoch in range(N_EPOCHS):
    # Shuffle indices
    indices = torch.randperm(num_embeddings)
    num_batches = num_embeddings // BATCH_SIZE

    for i in range(num_batches):
        # Get batch indices
        batch_indices = indices[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        # Get source and target batch embeddings
        src_batch = src_embeddings[batch_indices]
        tgt_batch = tgt_embeddings[batch_indices]

        # Train discriminator
        discriminator_optimizer.zero_grad()
        # Map source embeddings
        src_mapped = mapping(src_batch)
        # Discriminator outputs
        src_preds = discriminator(src_mapped.detach())
        tgt_preds = discriminator(tgt_batch)
        # Labels
        src_labels = torch.zeros(src_batch.size(0), 1).to(device)
        tgt_labels = torch.ones(tgt_batch.size(0), 1).to(device)
        # Loss
        d_loss_src = nn.functional.binary_cross_entropy_with_logits(src_preds, src_labels)
        d_loss_tgt = nn.functional.binary_cross_entropy_with_logits(tgt_preds, tgt_labels)
        d_loss = d_loss_src + d_loss_tgt
        # Backpropagation
        d_loss.backward()
        discriminator_optimizer.step()

        # Train mapping (generator)
        mapping_optimizer.zero_grad()
        # Map source embeddings
        src_mapped = mapping(src_batch)
        # Discriminator output
        src_preds = discriminator(src_mapped)
        # Labels (want discriminator to think mapped embeddings are target)
        src_labels = torch.ones(src_batch.size(0), 1).to(device)
        # Loss
        g_loss = nn.functional.binary_cross_entropy_with_logits(src_preds, src_labels)
        # Backpropagation
        g_loss.backward()
        mapping_optimizer.step()

    print(f'Epoch {epoch + 1}/{N_EPOCHS}, Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}')



Epoch 1/10, Discriminator Loss: 1.0607, Generator Loss: 1.6342
Epoch 2/10, Discriminator Loss: 0.9434, Generator Loss: 1.8573
Epoch 3/10, Discriminator Loss: 0.9147, Generator Loss: 1.7952
Epoch 4/10, Discriminator Loss: 0.9492, Generator Loss: 1.7618
Epoch 5/10, Discriminator Loss: 1.3018, Generator Loss: 2.1251
Epoch 6/10, Discriminator Loss: 1.0209, Generator Loss: 2.1135
Epoch 7/10, Discriminator Loss: 0.8881, Generator Loss: 2.3307
Epoch 8/10, Discriminator Loss: 0.8039, Generator Loss: 2.3603
Epoch 9/10, Discriminator Loss: 0.7237, Generator Loss: 3.0407
Epoch 10/10, Discriminator Loss: 0.7870, Generator Loss: 3.1606


In [24]:
W_adv = mapping.weight.data.cpu().numpy()

# Map source embeddings
en_mapped_embeddings = src_embeddings.cpu().numpy() @ W_adv.T

In [25]:
n_mapped_embeddings = src_embeddings.cpu().numpy() @ W_adv.T

# Normalize mapped embeddings
en_mapped_embeddings = en_mapped_embeddings / np.linalg.norm(en_mapped_embeddings, axis=1, keepdims=True)

In [1]:
def build_dictionary(src_emb, tgt_emb, src_words, tgt_words, k=1):
    src_emb_norm = src_emb / np.linalg.norm(src_emb, axis=1, keepdims=True)
    tgt_emb_norm = tgt_emb / np.linalg.norm(tgt_emb, axis=1, keepdims=True)
    similarities = src_emb_norm @ tgt_emb_norm.T
    nn_indices = np.argpartition(-similarities, range(k), axis=1)[:, :k]
    word_pairs = []
    for i, indices in enumerate(nn_indices):
        src_word = src_words[i]
        for idx in indices:
            tgt_word = tgt_words[idx]
            word_pairs.append((src_word, tgt_word))
    return word_pairs

# Build pseudo-dictionary
pseudo_dict = build_dictionary(en_mapped_embeddings, hi_embeddings.vectors, en_embeddings.index_to_key, hi_embeddings.index_to_key, k=1)

# Step 11: Refine the Mapping using Procrustes
def create_embedding_matrices(bilingual_dict, source_embeddings, target_embeddings):
    source_matrix = []
    target_matrix = []
    for src_word, tgt_word in bilingual_dict:
        if src_word in source_embeddings.key_to_index and tgt_word in target_embeddings.key_to_index:
            source_matrix.append(source_embeddings[src_word])
            target_matrix.append(target_embeddings[tgt_word])
    return np.array(source_matrix), np.array(target_matrix)

# Create matrices using pseudo-dictionary
X_train, Y_train = create_embedding_matrices(pseudo_dict, en_embeddings, hi_embeddings)

# Compute refined mapping
def compute_procrustes(X, Y):
    # Center the embeddings
    X_mean = X.mean(0)
    Y_mean = Y.mean(0)
    X -= X_mean
    Y -= Y_mean
    # Compute covariance matrix
    M = Y.T @ X
    # Singular Value Decomposition
    U, _, Vt = np.linalg.svd(M)
    # Compute orthogonal matrix W
    W = U @ Vt
    return W

W_refined = compute_procrustes(X_train, Y_train)

# Map English embeddings using refined mapping
en_mapped_embeddings = (en_embeddings.vectors - en_embeddings.vectors.mean(0)) @ W_refined.T + hi_embeddings.vectors.mean(0)

# Normalize mapped embeddings
en_mapped_embeddings = en_mapped_embeddings / np.linalg.norm(en_mapped_embeddings, axis=1, keepdims=True)

# Create KeyedVectors instance
en_mapped = KeyedVectors(vector_size=EMBEDDING_DIM)
en_mapped.add_vectors(en_embeddings.index_to_key, en_mapped_embeddings)

# Step 12: Compute CSLS Similarities
def compute_csls(src_emb, tgt_emb, k=10, batch_size=1024):
    src_avg_sim = np.zeros(src_emb.shape[0])
    tgt_avg_sim = np.zeros(tgt_emb.shape[0])

    # Compute source to target similarities in batches
    for i in range(0, src_emb.shape[0], batch_size):
        src_batch = src_emb[i:i+batch_size]
        sims = src_batch @ tgt_emb.T
        sorted_sims = np.sort(sims, axis=1)[:, -k:]
        src_avg_sim[i:i+batch_size] = np.mean(sorted_sims, axis=1)

    # Compute target to source similarities in batches
    for i in range(0, tgt_emb.shape[0], batch_size):
        tgt_batch = tgt_emb[i:i+batch_size]
        sims = tgt_batch @ src_emb.T
        sorted_sims = np.sort(sims, axis=1)[:, -k:]
        tgt_avg_sim[i:i+batch_size] = np.mean(sorted_sims, axis=1)

    return src_avg_sim, tgt_avg_sim

# Compute CSLS average similarities
src_avg_sim, tgt_avg_sim = compute_csls(en_mapped_embeddings, hi_embeddings.vectors, k=K_CSLS, batch_size=1024)

NameError: name 'en_mapped_embeddings' is not defined

In [ ]:
def precision_at_k_csls(source_embeddings, target_embeddings, test_dict, src_avg_sim, tgt_avg_sim, k=1, batch_size=1024):
    correct = 0
    total = 0
    src_vectors = source_embeddings.vectors
    tgt_vectors = target_embeddings.vectors

    # Build index mappings
    src_word2idx = source_embeddings.key_to_index
    tgt_word2idx = target_embeddings.key_to_index

    # Prepare test data
    test_src_indices = []
    test_tgt_indices = []
    for src_word, tgt_word in test_dict:
        if src_word in src_word2idx and tgt_word in tgt_word2idx:
            test_src_indices.append(src_word2idx[src_word])
            test_tgt_indices.append(tgt_word2idx[tgt_word])

    num_batches = (len(test_src_indices) + batch_size - 1) // batch_size

    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(test_src_indices))
        src_indices_batch = test_src_indices[batch_start:batch_end]
        tgt_indices_batch = test_tgt_indices[batch_start:batch_end]
        src_vecs = src_vectors[src_indices_batch]
        src_csls_sim = src_avg_sim[src_indices_batch]

        # Compute CSLS similarities
        sims = src_vecs @ tgt_vectors.T
        csls_sims = 2 * sims - src_csls_sim[:, None] - tgt_avg_sim[None, :]

        # For each source word in the batch
        for j in range(csls_sims.shape[0]):
            sims_row = csls_sims[j]
            top_k_indices = np.argpartition(-sims_row, range(k))[:k]
            if tgt_indices_batch[j] in top_k_indices:
                correct += 1
            total += 1

    precision = correct / total if total > 0 else 0
    return precision

# Evaluate
p_at_1_csls = precision_at_k_csls(en_mapped, hi_embeddings, test_dict, src_avg_sim, tgt_avg_sim, k=1, batch_size=1024)
p_at_5_csls = precision_at_k_csls(en_mapped, hi_embeddings, test_dict, src_avg_sim, tgt_avg_sim, k=5, batch_size=1024)

print(f'Unsupervised Alignment with CSLS - Precision@1: {p_at_1_csls:.4f}')
print(f'Unsupervised Alignment with CSLS - Precision@5: {p_at_5_csls:.4f}')